In [1]:
import numpy as np 
import pandas as pd 

!pip install fastai --upgrade
import os

import time
import matplotlib.pyplot as plt
from fastai.vision.all import *
%pip install torch
import torch
import numpy as np
import random
from sklearn.metrics import roc_curve, auc


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed(42)

In [4]:
path = r'C:\Users\Shekhar\OneDrive\Desktop\Capstone DS\Faultfindy\Faultfindy'
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=[*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
)
dls = data.dataloaders(path)

TypeError: 'NoneType' object is not iterable

In [ ]:
dls.show_batch(max_n=6)
print(f"Number of training examples: {len(dls.train_ds)}, Number of validation examples: {len(dls.valid_ds)}")

In [ ]:
metrics = [accuracy, RocAucBinary()]
def plot_roc_curve(learn, title="ROC Curve"):
    preds, targets = learn.get_preds()
    fpr, tpr, thresholds = roc_curve(targets, preds[:,1])
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
learn_resnet34 = vision_learner(dls, resnet34, metrics=metrics)
lr_resnet34 = learn_resnet34.lr_find()

In [ ]:
start_time_resnet34 = time.time()

learn_resnet34.fine_tune(10, base_lr=lr_resnet34.valley, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3)])

end_time_resnet34 = time.time()
training_time_resnet34 = end_time_resnet34 - start_time_resnet34
training_time_resnet34

In [ ]:
accuracy_resnet34 = learn_resnet34.validate()[1]
accuracy_resnet34

In [ ]:
interp_resnet34 = ClassificationInterpretation.from_learner(learn_resnet34)
interp_resnet34.plot_top_losses(k=6)
interp_resnet34.plot_confusion_matrix()
plt.show()

In [ ]:
plot_roc_curve(learn_resnet34, "ROC Curve for ResNet34")

In [ ]:
learn_resnet50 = vision_learner(dls, resnet50, metrics=metrics)
lr_resnet50 = learn_resnet50.lr_find()

In [ ]:
start_time_resnet50 = time.time()

learn_resnet50.fine_tune(10, base_lr=lr_resnet50.valley, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3)])

end_time_resnet50 = time.time()
training_time_resnet50 = end_time_resnet50 - start_time_resnet50
training_time_resnet50

In [ ]:
accuracy_resnet50 = learn_resnet50.validate()[1]
accuracy_resnet50

In [ ]:
interp_resnet50 = ClassificationInterpretation.from_learner(learn_resnet50)
interp_resnet50.plot_top_losses(k=6)
interp_resnet50.plot_confusion_matrix()
plt.show()

In [ ]:
plot_roc_curve(learn_resnet50, "ROC Curve for ResNet50")

In [ ]:
print(f"Training time for ResNet34: {training_time_resnet34:.2f} seconds")
print(f"Accuracy for ResNet34: {accuracy_resnet34:.4f}")
print(f"Training time for ResNet50: {training_time_resnet50:.2f} seconds")
print(f"Accuracy for ResNet50: {accuracy_resnet50:.4f}")

In [ ]:
learn_resnet34.export('resnet34_model.pkl')
learn_resnet50.export('resnet50_model.pkl')

In [ ]:
import time
import ipywidgets as widgets
from IPython.display import display
from fastai.vision.all import PILImage, load_learner
from PIL import UnidentifiedImageError
models = {'ResNet34': 'resnet34_model.pkl', 'ResNet50': 'resnet50_model.pkl'}
def load_model(change):
    global learn
    learn = load_learner(models[change.new])
vbox = widgets.VBox()
title = widgets.HTML('<h1>Tyre Defect Classifier</h1>')
description = widgets.HTML('<p>This tool classifies tyres as defective or good.</p>')
vbox.children += (title, description,)
model_selector = widgets.Dropdown(options=models.keys(), description='Model:')
model_selector.observe(load_model, names='value')
vbox.children += (model_selector,)
learn = load_learner(models[model_selector.value])
uploader = widgets.FileUpload()
vbox.children += (widgets.Label('Upload your image:'), uploader,)
predict_btn = widgets.Button(description="Predict", disabled=True, button_style='success')
vbox.children += (predict_btn,)
output_image = widgets.Output()
output_label = widgets.Label()
vbox.children += (output_image, output_label,)
def update_button_state(change):
    predict_btn.disabled = not bool(uploader.data)
uploader.observe(update_button_state, names='data')
def on_click_classify(change):
    output_image.clear_output()
    output_label.value = ''
    if uploader.data:
        try:
            img = PILImage.create(uploader.data[-1])
            with output_image:
                display(img.to_thumb(128, 128))
            start_time = time.time()
            pred, _, probs = learn.predict(img)
            end_time = time.time()
            output_label.value = f'Prediction: {pred}; Probability: {max(probs):.04f}; Time taken: {end_time - start_time:.2f} seconds'
        except UnidentifiedImageError:
            output_label.value = 'Error: Uploaded file is not a valid image. Please upload a valid image file.'
        except Exception as e:
            output_label.value = f'Error: {str(e)}'
            uploader.value.clear()
        uploader._counter = 0
        predict_btn.disabled = True
predict_btn.on_click(on_click_classify)
display(vbox)